In [1]:
# Set up JSON output syntax highlighting and other globals
from __future__ import print_function
from pygments import highlight
from pygments.lexers import JsonLexer, TextLexer
from pygments.formatters import HtmlFormatter
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime
import stix_notebook
import stix2viz
import re
from stix2.v21 import *
import os
import sys
sys.path.append(os.path.abspath('../firr'))
import identity_contact_info

NOW = datetime.now()

InteractiveShell.ast_node_interactivity = "all"

def json_print(inpt):
    string = str(inpt)
    formatter = HtmlFormatter()
    if string[0] in ['{', '[']:
        lexer = JsonLexer()
        viz_graph = stix2viz.display(inpt).data
        viz_graph = re.sub(r'(<svg.* style=")', r'\1border: 2px solid #ababab;', viz_graph, 1)
    else:
        lexer = TextLexer()
        viz_graph = ""
    return HTML(viz_graph + '<style type="text/css">{}</style>{}'.format(
                formatter.get_style_defs('.highlight'),
                highlight(string, lexer, formatter)))

def print_graph(stix_objs):
    output = ',\n'.join(obj.serialize(pretty=True) for obj in stix_objs)
    if len(stix_objs) > 1:
        output = '[' + output + ']'
    return json_print(output)

globals()['print'] = json_print


**Scenario: individual (not a member of organization), reporting on behalf of impacted entity.**

Section #3.i. Incident Determination

In [2]:
import datetime
NOW = datetime.datetime.now()
INCIDENT_ID = "incident--6e3b19e1-645b-4323-8c15-1fd04b2f6d3d"
obj_list = []

irf_system = Identity(x_custom="IRF system",
    name="IRF",
    description="Incident Reporting Form (IRF) system",
    identity_class="system",
    allow_custom=True,
)
obj_list.append(irf_system)

toplevel_org_email = EmailAddress(
    value="soc@organization1.com",
)
obj_list.append(toplevel_org_email)

toplevel_org = Identity(
    x_comment="#1.b. Organization",
    name="Organization1",
    description="An example entity that was impacted.",
    identity_class="critical-infrastructure-or-private",
    roles=["cleared-dod-contractor"],
    sectors=["commercial", "defense", "defense-industrial-base-primary", "chemical-secondary"],
    external_references=[
        {
          "source_name": "Preferred Website",
          "external_id": "hxxp://organization1.com"
        },
        {
          "source_name": "Internal Tracking Number",
          "external_id": "2222"
        },
        {
          "source_name": "NAICS ID",
          "external_id": "3333"
        },
        {
          "source_name": "GSA UEI",
          "external_id": "4444"
        },
    ],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "email_addresses": [
                {
                    "description": "This is the prefered email address of the organization.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": toplevel_org_email.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(toplevel_org)

reporter_email = EmailAddress(
    value="jane.doe@example.com",
    display_name="Jane Doe"
)
obj_list.append(reporter_email)

reporter_social_media = UserAccount(
    account_login="@JaneDoe",
    account_type="Twitter"
)
obj_list.append(reporter_social_media)

reporter = Identity(
    x_comment="#1.3. Reporter",
    created_by_ref=irf_system,
    name="Jane K. Doe, PhD",
    description="An example individual reporting an incident on behalf of a different entity.",
    identity_class="individual",
    sectors=["commercial", "technology"],
    roles=["reporter"],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "first_name": "Jane",
            "middle_name": "K.",
            "last_name": "Doe",
            "suffix": "PhD",
            "contact_numbers": [
                {
                    "contact_number_type": "unclassified",
                    "contact_number": "(111) 111-1111"
                }
            ],
            "email_addresses": [
                {
                    "digital_contact_type": "unclassified",
                    "email_address_ref": reporter_email.id
                }
            ],
            "social_media_accounts": [
                {
                    "digital_contact_type": "personal",
                    "user_account_ref": reporter_social_media.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(reporter)

rel1 = Relationship(
    relationship_type='contact-for',
    source_ref=reporter,
    target_ref=INCIDENT_ID,
)
obj_list.append(rel1)

rel2 = Relationship(
    relationship_type='reporting-on-behalf-of',
    source_ref=reporter,
    target_ref=toplevel_org,
)
obj_list.append(rel2)

sub_entity_email = EmailAddress(
    value="point1@organization1.com",
)
obj_list.append(sub_entity_email)

sub_entity = Identity(
    x_comment="#1.a.3.E.1.a.2. & #1.b.6. Impacted Sub-Entity",
    name="Organization1.1",
    description="The sub-entity that was impacted.",
    external_references=[
        {
          "source_name": "Sub-Entity Tracking Number",
          "external_id": "12121212"
        },
        {
          "source_name": "Preferred Website",
          "external_id": "hxxp://point1.organization1.com"
        }
    ],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "email_addresses": [
                {
                    "description": "This is the prefered email address of the sub-entity.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": sub_entity_email.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(sub_entity)

rel3 = Relationship(
    relationship_type='unit-of',
    source_ref=sub_entity,
    target_ref=toplevel_org,
)
obj_list.append(rel3)

org_email = EmailAddress(
    value="soc@organization1.com"
)
obj_list.append(org_email)

impacted_entity_poc_email = EmailAddress(
    value="sam.smith@organization1.com",
    display_name="Sam Smith"
)
obj_list.append(impacted_entity_poc_email)

impacted_entity_poc_social_media = UserAccount(
    account_login="@SamSmith",
    account_type="Twitter"
)
obj_list.append(impacted_entity_poc_social_media)

impacted_entity_poc = Identity(
    x_comment="#1.a.4.D.1.b. Impacted Entity Contact Info",
    name="Sam S. Smith",
    description="Impacted entity point of contact.",
    identity_class="organization",
    sectors=["commercial", "defense"],
    roles=["director"],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "first_name": "Sam",
            "middle_name": "S.",
            "last_name": "Smith",
            "contact_numbers": [
                {
                    "contact_number_type": "unclassified",
                    "contact_number": "(222) 222-2222"
                }
            ],
            "email_addresses": [
                {
                    "description": "This is the email address of the Point of Contact at the organization.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": impacted_entity_poc_email.id
                },
                {
                    "description": "This is the prefered email address of the organization.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": org_email.id
                }
            ],
            "social_media_accounts": [
                {
                    "digital_contact_type": "unclassified",
                    "user_account_ref": impacted_entity_poc_social_media.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(impacted_entity_poc)

impacted_entity_poc_address = Location(
    street_address="0000 Example St.",
    city="Arlington",
    administrative_area="US-VA",
    country="us",
    postal_code="000000"
)
obj_list.append(impacted_entity_poc_address)

rel4 = Relationship(
    relationship_type='located-at',
    source_ref=impacted_entity_poc,
    target_ref=impacted_entity_poc_address,
)
obj_list.append(rel4)

rel5 = Relationship(
    relationship_type='employed-at',
    source_ref=impacted_entity_poc,
    target_ref=toplevel_org,
)
obj_list.append(rel5)

rel6 = Relationship(
    relationship_type='contact-for',
    source_ref=impacted_entity_poc,
    target_ref=INCIDENT_ID,
)
obj_list.append(rel6)

org_247_poc_email = EmailAddress(
    value="mary.johnson@organization1.com",
    display_name="Mary Johnson"
)
obj_list.append(org_247_poc_email)

org_247_poc_social_media = UserAccount(
    account_login="@MaryJohnson",
    account_type="Twitter"
)
obj_list.append(org_247_poc_social_media)

org_247_poc = Identity(
    x_comment="#1.a.4.D.1.b.7. Organization 24/7 contact",
    name="Mary M. Johnson",
    description="24/7 operations center point of contact.",
    identity_class="individual",
    sectors=["commercial", "defense"],
    roles=["24-7-operations-center-poc"],
    external_references=[
        {
          "source_name": "timezone",
          "external_id": "EST"
        }
    ],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "first_name": "Mary",
            "middle_name": "M.",
            "last_name": "Johnson",
            "contact_numbers": [
                {
                    "contact_number_type": "unclassified",
                    "contact_number": "(666) 666-6666"
                }
            ],
            "email_addresses": [
                {
                    "description": "This is the email address of the 24/7 Point of Contact at the organization.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": org_247_poc_email.id
                }
            ],
            "social_media_accounts": [
                {
                    "digital_contact_type": "personal",
                    "user_account_ref": org_247_poc_social_media.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(org_247_poc)

org_247_poc_address = Location(
    street_address="1111 Something Road",
    city="Arlington",
    administrative_area="US-VA",
    country="us",
    postal_code="000000"
)
obj_list.append(org_247_poc_address)

rel4 = Relationship(
    relationship_type='located-at',
    source_ref=org_247_poc,
    target_ref=org_247_poc_address,
)
obj_list.append(rel4)

rel7 = Relationship(
    relationship_type='employed-at',
    source_ref=org_247_poc,
    target_ref=toplevel_org,
)
obj_list.append(rel7)

rel8 = Relationship(
    relationship_type='contact-for',
    source_ref=org_247_poc,
    target_ref=INCIDENT_ID,
)
obj_list.append(rel8)

org_address = Location(
    street_address="1234 Org One Ave",
    city="Arlington",
    administrative_area="US-VA",
    country="us",
    postal_code="000000",
    external_references=[
        {
          "source_name": "CSAT Facility ID",
          "external_id": "111111"
        },
        {
          "source_name": "EPA FacID",
          "external_id": "33334444"
        }
    ]
)
obj_list.append(org_address)

rel9 = Relationship(
    relationship_type='located-at',
    source_ref=toplevel_org,
    target_ref=org_address,
)
obj_list.append(rel9)

org_cage_code = Location(
    street_address="4321 Org Street",
    city="Burlington",
    administrative_area="US-MA",
    country="us",
    postal_code="01805",
    external_references=[
        {
          "source_name": "CAGE",
          "external_id": "5555",
          "description": "Facility Clearance Level: Secret\n Cleared contractor"
        }
    ]
)
obj_list.append(org_cage_code)

rel10 = Relationship(
    relationship_type='related-to',
    source_ref=org_cage_code,
    target_ref=toplevel_org,
)
obj_list.append(rel10)

usg_contracting_poc_email = EmailAddress(
    value="james.kim@organization1.com",
    display_name="James Kim"
)
obj_list.append(usg_contracting_poc_email)

usg_contracting_poc = Identity(
    x_comment="#1.b.5.J. USG Contracting Point of Contact",
    name="James Kim",
    description="USG Contracting point of contact.",
    identity_class="individual",
    sectors=["commercial", "defense"],
    roles=["usg-contracting-poc", "lead-contracts-manager"],
    external_references=[
        {
          "source_name": "timezone",
          "external_id": "EST"
        },
        {
          "source_name": "contract number",
          "external_id": "1A1B1C",
          "description": "Prime Contractor"
        },
        {
          "source_name": "contract clearance level",
          "external_id": "unclassified"
        }
    ],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "first_name": "James",
            "last_name": "Kim",
            "contact_numbers": [
                {
                    "contact_number_type": "unclassified",
                    "contact_number": "(777) 777-7777"
                }
            ],
            "email_addresses": [
                {
                    "description": "This is the email address of the USG Contracting Point of Contact.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": usg_contracting_poc_email.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(usg_contracting_poc)

usg_contracting_poc_address = Location(
    street_address="412 Pine St.",
    city="Arlington",
    administrative_area="US-VA",
    country="us",
    postal_code="000000"
)
obj_list.append(usg_contracting_poc_address)

rel11 = Relationship(
    relationship_type='located-at',
    source_ref=usg_contracting_poc,
    target_ref=usg_contracting_poc_address,
)
obj_list.append(rel11)

rel12 = Relationship(
    relationship_type='employed-at',
    source_ref=usg_contracting_poc,
    target_ref=toplevel_org,
)
obj_list.append(rel12)

rel13 = Relationship(
    relationship_type='contact-for',
    source_ref=usg_contracting_poc,
    target_ref=INCIDENT_ID,
)
obj_list.append(rel13)

usg_admin_poc_email = EmailAddress(
    value="olivia.williams@organization1.com",
    display_name="Olivia Williams"
)
obj_list.append(usg_admin_poc_email)

usg_admin_poc = Identity(
    x_comment="#1.b.5.K. USG Administrative Contracting Officer Point of Contact",
    name="Olivia Williams",
    description="USG Administrative Contracting point of contact.",
    identity_class="individual",
    sectors=["commercial", "defense"],
    roles=["usg-administrative-contracting-poc", "lead-contracts-manager"],
    external_references=[
        {
          "source_name": "timezone",
          "external_id": "EST"
        }
    ],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "first_name": "Olivia",
            "last_name": "Williams",
            "contact_numbers": [
                {
                    "contact_number_type": "unclassified",
                    "contact_number": "(888) 888-8888"
                }
            ],
            "email_addresses": [
                {
                    "description": "This is the email address of the USG Administrative Contracting Officer.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": usg_admin_poc_email.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(usg_admin_poc)

rel15 = Relationship(
    relationship_type='employed-at',
    source_ref=usg_admin_poc,
    target_ref=toplevel_org,
)
obj_list.append(rel15)

rel16 = Relationship(
    relationship_type='contact-for',
    source_ref=usg_admin_poc,
    target_ref=INCIDENT_ID,
)
obj_list.append(rel16)

usg_pm_poc_email = EmailAddress(
    value="patrick.brown@organization1.com",
    display_name="Patrick Brown"
)
obj_list.append(usg_pm_poc_email)

usg_pm_poc = Identity(
    x_comment="#1.b.5.L. USG Program Manager Point of Contact",
    name="Patrick Brown",
    description="USG Program Manager point of contact.",
    identity_class="individual",
    sectors=["commercial", "defense"],
    roles=["usg-program-manager-poc", "govt-program-manager"],
    external_references=[
        {
          "source_name": "timezone",
          "external_id": "EST"
        },
        {
          "source_name": "contract number",
          "external_id": "1A1B1C",
          "description": "Prime Contractor"
        },
        {
          "source_name": "contract clearance level",
          "external_id": "unclassified"
        }
    ],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "first_name": "Patrick",
            "last_name": "Brown",
            "contact_numbers": [
                {
                    "contact_number_type": "unclassified",
                    "contact_number": "(999) 999-9999"
                }
            ],
            "email_addresses": [
                {
                    "description": "This is the email address of the USG Program Manager.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": usg_pm_poc_email.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(usg_pm_poc)

usg_pm_poc_address = Location(
    street_address="32 Lakeside Ave.",
    city="Arlington",
    administrative_area="US-VA",
    country="us",
    postal_code="000000"
)
obj_list.append(usg_pm_poc_address)

rel17 = Relationship(
    relationship_type='located-at',
    source_ref=usg_pm_poc,
    target_ref=usg_pm_poc_address,
)
obj_list.append(rel17)

rel18 = Relationship(
    relationship_type='employed-at',
    source_ref=usg_pm_poc,
    target_ref=toplevel_org,
)
obj_list.append(rel18)

rel19 = Relationship(
    relationship_type='contact-for',
    source_ref=usg_pm_poc,
    target_ref=INCIDENT_ID,
)
obj_list.append(rel19)

cisa_assistance = CourseOfAction(
    x_comment="#1.c.7. Assistance - CISA",
    name="requesting-CISA-onsite-assistance",
    labels=[
        "logs-available"
    ],
    allow_custom=True,
)
obj_list.append(cisa_assistance)

contracted_assistance = CourseOfAction(
    x_comment="#1.c.8. Contracted Assistance",
    name="contracting-out-assistance",
    labels=[
        "contracted-analysis-shareable",
        "log-data-shareable"
    ],
    allow_custom=True,
)
obj_list.append(contracted_assistance)

contracted_assistance_entity = Identity(
    x_comment="#1.c.8. Contracted Assistance",
    name="Incident Assistance Contracted Entity",
    description="The entity contracted out for assistance with the reported incident.",
    allow_custom=True,
)
obj_list.append(contracted_assistance_entity)

rel20 = Relationship(
    relationship_type='contracted-for',
    source_ref=contracted_assistance_entity,
    target_ref=contracted_assistance,
)
obj_list.append(rel20)

report_to_congress = CourseOfAction(
    x_comment="#1.d.11.A. Reported to Congress",
    name="Reported to Congress",
    allow_custom=True,
)
obj_list.append(report_to_congress)

congress_committee = Identity(
    x_comment="#1.d.11.A.2.a. Congressional Committee reported to",
    name="House Homeland Security Committee",
    identity_class="organization",
    allow_custom=True,
)
obj_list.append(congress_committee)

congress_poc_email = EmailAddress(
    value="ben.franklin@congress.gov",
    display_name="Benjamin Franklin"
)
obj_list.append(congress_poc_email)

congress_poc = Identity(
    x_comment="#1.d.11.A.2.a.2. Congressional Point of Contact",
    name="Benjamin Franklin",
    description="Congressional point of contact.",
    identity_class="individual",
    sectors=["government-national"],
    roles=["congressional-poc", "staffer"],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "first_name": "Patrick",
            "last_name": "Brown",
            "contact_numbers": [
                {
                    "contact_number_type": "unclassified",
                    "contact_number": "(343) 434-3434"
                }
            ],
            "email_addresses": [
                {
                    "description": "Email address of the Congressional Point of Contact.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": congress_poc_email.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(congress_poc)

rel21 = Relationship(
    relationship_type='related-to',
    source_ref=congress_poc,
    target_ref=congress_committee,
)
obj_list.append(rel21)

rel22 = Relationship(
    relationship_type='contact-for',
    source_ref=congress_poc,
    target_ref=INCIDENT_ID,
)
obj_list.append(rel22)

intrusion_infra = Infrastructure(
    name="Defense and Weapons Systems",
    infrastructure_types=[
        "non-malicious",
        "defense-and-weapons-systems"
    ]
)
obj_list.append(intrusion_infra)

breach_pii_observed = URL(
    value="https://example.com/research/index.html"
)
obj_list.append(breach_pii_observed)

impacted_system = Infrastructure(
    x_comment="#3.i.1. Impacted System",
    name="User database",
    description="SomeDB version 2.1.3",
    infrastructure_types=[
        "non-malicious",
        "fisma-system",
        "general-support-system",
        "hva",
        "hva-tier-1",
        "hva-connections-internal",
        "dod-system",
        "intelligence-community",
        "shared-with-gov-agency"
    ],
    allow_custom=True,
)
obj_list.append(impacted_system)

impacted_system_other = Infrastructure(
    x_comment="#3.i.1.A.4.b.2.ii. Impacted systems - other HVAs",
    name="SomeApp",
    description="SomeApp version 1.1. Supports Service A.",
    infrastructure_types=[
        "non-malicious",
        "fisma-system",
        "general-support-system",
        "hva",
        "hva-tier-1",
        "hva-connections-external",
        "dod-system",
        "intelligence-community",
        "classified-information-source"
    ],
    allow_custom=True,
)
obj_list.append(impacted_system_other)

rel23 = Relationship(
    relationship_type='communicates-with',
    source_ref=impacted_system,
    target_ref=impacted_system_other,
)
obj_list.append(rel23)

impacted_system_community = Identity(
    x_comment="#3.i.1.A.4.b.2.ii.1.i.A. Impacted systems - other HVAs - supported communities",
    name="System A users",
    identity_class="class",
    allow_custom=True,
)
obj_list.append(impacted_system_community)

rel24 = Relationship(
    relationship_type='supports',
    source_ref=impacted_system_other,
    target_ref=impacted_system_community,
)
obj_list.append(rel24)

other_hva_poc_email = EmailAddress(
    value="amelia.garcia@example.com",
    display_name="Amelia Garcia"
)
obj_list.append(other_hva_poc_email)

other_hva_poc = Identity(
    x_comment="#3.i.1.A.4.b.2.ii.1.ii. Impacted systems - other HVAs - contact information",
    name="Amelia Garcia",
    description="impacted system other HVA Point of Contact.",
    identity_class="individual",
    roles=["Support Engineer", "system-owner"],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "first_name": "Amelia",
            "last_name": "Garcia",
            "contact_numbers": [
                {
                    "contact_number_type": "unclassified",
                    "contact_number": "(565) 656-5656"
                }
            ],
            "email_addresses": [
                {
                    "description": "Email address of the impacted system other HVA Point of Contact.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": other_hva_poc_email.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(other_hva_poc)

rel25 = Relationship(
    relationship_type="owner-of",
    source_ref=other_hva_poc,
    target_ref=impacted_system_other,
)
obj_list.append(rel25)

impacted_system_poc_email = EmailAddress(
    value="madison.miller@organization1.com",
    display_name="Madison Miller"
)
obj_list.append(impacted_system_poc_email)

impacted_system_poc = Identity(
    x_comment="#3.i.1.A.4.c. Impacted system owner",
    name="Madison Miller",
    description="impacted system owner",
    identity_class="individual",
    roles=["Systems Lead", "system-owner"],
    extensions={
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "first_name": "Madison",
            "last_name": "Miller",
            "contact_numbers": [
                {
                    "contact_number_type": "unclassified",
                    "contact_number": "(787) 878-7878"
                }
            ],
            "email_addresses": [
                {
                    "description": "Email address of the impacted system owner.",
                    "digital_contact_type": "unclassified",
                    "email_address_ref": impacted_system_poc_email.id
                }
            ]
        }
    },
    allow_custom=True,
)
obj_list.append(impacted_system_poc)

rel26 = Relationship(
    relationship_type="owner-of",
    source_ref=impacted_system_poc,
    target_ref=impacted_system,
)
obj_list.append(rel26)

impacted_system_cage_code = Location(
    x_comment="#3.i.1.A.8. Impacted system CAGE code",
    street_address="853 Lexington Ave.",
    city="Christiansburg",
    administrative_area="US-VA",
    country="us",
    postal_code="24073",
    external_references=[
        {
          "source_name": "CAGE",
          "external_id": "1234",
        }
    ],
    allow_custom=True,
)
obj_list.append(impacted_system_cage_code)

rel27 = Relationship(
    relationship_type='located-at',
    source_ref=impacted_system,
    target_ref=impacted_system_cage_code,
)
obj_list.append(rel27)

rel28 = Relationship(
    relationship_type='caused-exposure-of',
    source_ref=INCIDENT_ID,
    target_ref=impacted_system,
)
obj_list.append(rel28)

exposure_system = Infrastructure(
    x_comment="#3.i.2.A.4. System exposure occured on",
    name="System exposure occured on",
    allow_custom=True,
)
obj_list.append(exposure_system)

rel29 = Relationship(
    relationship_type='communicates-with',
    source_ref=impacted_system,
    target_ref=exposure_system,
)
obj_list.append(rel29)

incident = Incident(
    name="IRF-01",
    id=INCIDENT_ID,
    extensions={
        "extension-definition--ef765651-680c-498d-9894-99799f2fa126": {
            "extension_type": "property-extension",
            "description": "A description of the nature of the activity.",
            "determination": "suspected",
            "investigation_status": "under-investigation",
            "labels": [
                "tracked-by-us-govt",
                "national-media-coverage",
                "tracked-by-foreign-state-adversaries",
                "tracked-by-foreign-nonstate-adversaries",
                "incident-discussed-online",
                "shared-with-federal-agency",
            ],
            "incident_types": [
                "imminent-cia-violation",
                "actual-cia-violation",
                "imminent-policy-violation",
                "actual-policy-violation",
                "potential-loss-of-control",
                "potential-compromised-system",
                "potential-unauthorized-release",
                "potential-unauthorized-access",
                "loss-of-control",
                "compromised-system",
                "unauthorized-user",
                "authorized-user",
            ],
            "attacker_activities": [
                {
                    "x_comment": "#1.d.12.A.1.",
                    "outcome": "successful",
                    "description": "Observed network intrusion or penetration",
                    "impacted_refs": [intrusion_infra.id],
                },
                {
                    "x_comment": "#1.d.12.A.1.",
                    "outcome": "successful",
                    "description": "Observed access to data storage or file systems",
                    "impacted_refs": [intrusion_infra.id],
                },
                {
                    "x_comment": "#1.d.12.A.1.",
                    "outcome": "successful",
                    "description": "Observed access to control systems",
                    "impacted_refs": [intrusion_infra.id],
                },
                {
                    "x_comment": "#1.d.12.A.3.a.1.",
                    "outcome": "unknown",
                    "description": "Human-enabled or close-access operations"
                },
                {
                    "x_comment": "#1.d.12.A.3.a.3.",
                    "outcome": "unknown",
                    "description": "Custom, tailored, or unique tool(s) used"
                },
                {
                    "x_comment": "#1.d.12.A.3.b.2.",
                    "outcome": "unknown",
                    "description": "Specificity of data or information stolen or manipulated"
                },
                {
                    "x_comment": "#1.d.12.A.3.b.4.",
                    "outcome": "unknown",
                    "description": "Aligned with known foreign government policies"
                },
                {
                    "x_comment": "#2.e. Breach Determination",
                    "outcome": "successful",
                    "goal": "Distribute PII",
                    "observed_refs": [breach_pii_observed.id],
                },
                {
                    "x_comment": "#3.i.1. Incident Determination - Impacted systems",
                    "description": "Some technical details bla bla bla",
                    "outcome": "unknown",
                    "impacted_refs": [impacted_system.id],
                },
                {
                    "x_comment": "#3.i.1.A.4.b.2.ii.2. Impacted systems - other HVAs",
                    "description": "Some other technical details bla bla bla",
                    "outcome": "unknown",
                    "impacted_refs": [impacted_system_other.id],
                },
                {
                    "x_comment": "#3.i.2.A.5. root cause of classified exposure",
                    "description": "Some technical details etc.",
                    "outcome": "successful",
                    "goal": "take classified data",
                    "impacted_refs": [exposure_system.id],
                },
            ],
            "defender_activities": [
                {
                    "x_comment": "#1.c.7.",
                    "activity_type": "request-assistance",
                    "outcome": "unknown",
                    "description": "Requesting CISA on-site assistance.",
                    "start_time": NOW,
                    "end_time": NOW,
                    "identity_refs": "identity--b3bca3c2-1f3d-4b54-b44f-dac42c3a8f01", # CISA ID
                    "object_ref": cisa_assistance.id
                },
                {
                    "x_comment": "#1.c.8.",
                    "activity_type": "external-support",
                    "outcome": "unknown",
                    "description": "Contracted out assistance.",
                    "start_time": NOW,
                    "end_time": NOW,
                    "identity_refs": [contracted_assistance_entity.id],
                    "object_ref": contracted_assistance.id
                },
                {
                    "x_comment": "#1.d.10.",
                    "activity_type": "upgrade-to-major-incident",
                    "outcome": "successful",
                    "description": "Some additional context about upgrading to major incident.",
                    "start_time": NOW,
                    "end_time": NOW,
                },
                {
                    "x_comment": "#1.d.11.A. Reported to Congress",
                    "activity_type": "reported-to-congress",
                    "outcome": "successful",
                    "description": "Reported to Congress.",
                    "start_time": "2022-05-02T07:00:00Z",
                    "end_time": "2022-05-02T07:00:00Z",
                    "identity_refs": [congress_committee.id],
                    "object_ref": report_to_congress.id
                },
                {
                    "x_comment": "#2.f.6. Impacted Individuals notified by Email",
                    "activity_type": "notification-email",
                    "outcome": "successful",
                    "description": "Notified 10 individuals",
                },
                {
                    "x_comment": "#2.f.6. Impacted Individuals notified by Verbal",
                    "activity_type": "notification-verbal",
                    "outcome": "successful",
                    "description": "Notified 10 individuals",
                },
                {
                    "x_comment": "#2.f.6. Impacted Individuals notified by Parcel",
                    "activity_type": "notification-parcel",
                    "outcome": "successful",
                    "description": "Notified 5 individuals",
                },
                {
                    "x_comment": "#2.f.7. Impacted Individuals services provided",
                    "activity_type": "provided-identity-monitoring",
                    "outcome": "successful",
                    "description": "Provided to 8 individuals",
                },
                {
                    "x_comment": "#2.f.7. Impacted Individuals services provided",
                    "activity_type": "provided-credit-monitoring",
                    "outcome": "successful",
                    "description": "Provided to 9 individuals",
                },
                {
                    "x_comment": "#2.f.7. Impacted Individuals services provided",
                    "activity_type": "provided-identity-theft-insurance",
                    "outcome": "successful",
                    "description": "Provided to 1 individuals",
                },
                {
                    "x_comment": "#3.i.2.A.1. Exposure of classified info",
                    "activity_type": "identiy-item-classified-material",
                    "outcome": "successful",
                    "description": "source material x",
                },
                {
                    "x_comment": "#3.i.2.A.2. Exposure of classified info appeal",
                    "activity_type": "classification-appeal",
                    "outcome": "successful",
                    "start_time": "2022-05-02T07:00:00Z",
                    "start_time_fidelity": "day",
                    "end_time": "2022-05-10T07:00:00Z",
                    "end_time_fidelity": "day",
                    "object_ref": impacted_system.id
                },
                {
                    "x_comment": "#3.i.3. Activity detected",
                    "activity_type": "detected",
                    "outcome": "successful",
                    "start_time": "2022-05-02T07:00:00Z",
                    "start_time_fidelity": "hour",
                    "end_time": "2022-05-10T07:00:00Z",
                    "end_time_fidelity": "hour",
                    "impacted_refs": [impacted_system.id],
                },
                {
                    "x_comment": "#3.i.5. Incident declared",
                    "activity_type": "declared",
                    "outcome": "successful",
                    "start_time": "2022-05-02T07:00:00Z",
                    "start_time_fidelity": "hour",
                    "end_time": "2022-05-10T07:00:00Z",
                    "end_time_fidelity": "hour",
                },
            ],
            "confidentiality_impacts": [
                {
                    "x_comment": "#1.d.9.H3.",
                    "criticality": 75,
                    "loss_type": "confirmed-loss",
                    "information_type": "pii",
                    "record_count": 100000,
                    "description": "unauthorized-exfiltration"
                },
                {
                    "x_comment": "#1.d.9.H4.",
                    "criticality": 25,
                    "loss_type": "confirmed-loss",
                    "information_type": "pii",
                    "record_count": 100000,
                    "description": "unauthorized-access"
                },
                {
                    "x_comment": "#1.d.12.A.2.d.",
                    "loss_type": "confirmed-loss",
                    "information_type": "proprietary"
                },
                {
                    "x_comment": "#2.e. Breach Determination",
                    "loss_type": "confirmed-loss",
                    "description": "Non-Cyber PII: some additional info about the PII",
                    "information_type": "pii"
                },
                {
                    "x_comment": "#2.g.8.A.1.",
                    "loss_type": "confirmed-loss",
                    "record_count": 25,
                    "description": "known",
                    "information_type": "social-security-number"
                },
                {
                    "x_comment": "#2.g.8.A.11.",
                    "loss_type": "suspected-loss",
                    "record_count": 15,
                    "description": "known",
                    "information_type": "passport-number"
                },
                {
                    "x_comment": "#2.g.8.A.15.",
                    "loss_type": "confirmed-loss",
                    "record_count": 25,
                    "description": "known",
                    "information_type": "employee-identification-number"
                },
                {
                    "x_comment": "#2.g.8.B.1.",
                    "loss_type": "confirmed-loss",
                    "record_count": 25,
                    "description": "known",
                    "information_type": "name"
                },
                {
                    "x_comment": "#2.g.8.F.8.",
                    "loss_type": "confirmed-loss",
                    "record_count": 25,
                    "description": "known",
                    "information_type": "personnel-files"
                },
                {
                    "x_comment": "#3.i.2.A.6. File types exposed",
                    "loss_type": "confirmed-loss",
                    "record_count": 5,
                    "description": ".pdf",
                    "information_type": "classified-material"
                },
            ],
            "external_impacts": [
                {
                    "x_comment": "#1.d.9.E.1.",
                    "impact_type": "national-security",
                    "criticality": 75,
                },
                {
                    "x_comment": "#1.d.9.E.2.",
                    "impact_type": "foreign-relations",
                    "criticality": 75,
                },
                {
                    "x_comment": "#1.d.9.E.3.",
                    "impact_type": "economic",
                    "criticality": 25,
                },
                {
                    "x_comment": "#1.d.9.E.4.",
                    "impact_type": "public-confidence",
                    "criticality": 25,
                },
                {
                    "x_comment": "#1.d.9.E.5.",
                    "impact_type": "civil-liberties",
                    "criticality": 25,
                },
                {
                    "x_comment": "#1.d.9.E.6.",
                    "impact_type": "public-health",
                    "criticality": 25,
                },
                {
                    "x_comment": "#1.d.12.A.2.a.",
                    "impact_type": "foreign-domestic-policy",
                },
                {
                    "x_comment": "#1.d.12.A.2.b.",
                    "impact_type": "government-operations",
                },
                {
                    "x_comment": "#1.d.12.A.2.c.",
                    "impact_type": "financial-systems",
                },
                {
                    "x_comment": "#1.d.12.A.2.e.",
                    "impact_type": "physical-damage",
                },
                {
                    "x_comment": "#1.d.12.A.2.f.",
                    "impact_type": "reputation",
                },
                {
                    "x_comment": "#3.i.7. Coast Guard facility/vessel involved",
                    "impact_type": "coast-guard-facility",
                    "description": "Coast Guard facility type A",
                },
                {
                    "x_comment": "#3.i.8. pipelines and facilities ",
                    "impact_type": "pipelines-infrastructure",
                    "description": "text describing a natural gas pipeline, including its location and identifying information",
                },
            ],
            "integrity_impacts": [
                {
                    "x_comment": "#1.d.9.H1.",
                    "criticality": 25,
                    "alteration": "full-modification",
                    "information_type": "pii",
                    "record_count": 100000
                },
                {
                    "x_comment": "#1.d.9.H2.",
                    "criticality": 75,
                    "alteration": "full-destruction",
                    "information_type": "pii",
                    "record_count": 100000
                },
                {
                    "x_comment": "#2.h.9.A.1.",
                    "alteration": "partial-modification",
                    "information_type": "social-security-number",
                    "description": "known",
                    "record_count": 25
                },
                {
                    "x_comment": "#2.h.9.A.15.",
                    "alteration": "partial-modification",
                    "information_type": "employee-identification-number",
                    "description": "known",
                    "record_count": 25
                },
                {
                    "x_comment": "#2.h.9.F.8.",
                    "alteration": "potential-modification",
                    "information_type": "personnel-files",
                    "description": "approximate",
                    "record_count": 25
                },
            ],
            "impacted_entity_counts": {
                "individual": 25
            },
        }
    },
    allow_custom=True,
)
obj_list.append(incident)

print_graph(obj_list)